In [1]:
!pip3 install pandas
!pip3 install PyArrow 

In [2]:
%env S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
%env PASSWORD_SQL=PASSWORD
%env USER_SQL=USER

env: S3ADRESS=s3a://daniel-3cco-poupay-poupay-bucket-estruturados
env: PASSWORD_SQL=PASSWORD
env: USER_SQL=USER


In [3]:
import os

S3_address = os.environ['S3ADRESS']
password_sql = os.environ['PASSWORD_SQL']
user_sql = os.environ['USER_SQL']

In [4]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.pandas as ps

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).config("spark.driver.memory", "15g").getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a85235d7-63ae-43bb-9849-14f5314a1540;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 387ms :: artifacts dl 21ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [5]:
df = spark.read.option('delimiter', ',')\
    .option('header', 'true')\
    .csv(f'{S3_address}/dados-brutos/Value_of_Production_E_All_Data_(Normalized).csv')

22/06/03 00:00:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
df.show()

+---------+-----------+---------+-------------------+------------+--------------------+---------+----+-----------+------------+----+
|Area Code|       Area|Item Code|               Item|Element Code|             Element|Year Code|Year|       Unit|       Value|Flag|
+---------+-----------+---------+-------------------+------------+--------------------+---------+----+-----------+------------+----+
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1976|1976|1000 Int. $|49550.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1977|1977|1000 Int. $|45505.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1978|1978|1000 Int. $|60673.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gross Production ...|     1979|1979|1000 Int. $|53089.000000|  Fc|
|        2|Afghanistan|      221|Almonds, with shell|         152|Gro

In [7]:
dataframe_brazil = df.filter(df.Area == 'Brazil')

In [16]:
# from pyspark.sql import functions as F

# dataframe_brazil = dataframe_brazil.withColumn('Item',F.when(dataframe_brazil['Item'] == 'Meat, cattle', 'Carne, Gado')
#                                                .when(dataframe_brazil['Item'] == 'Soybeans', 'Soja')
#                                                .otherwise(F.col("Item")))

In [17]:
dataframe_brazil.show()

+---------+------+---------+------+------------+--------------------+---------+----+-----------+------------+----+
|Area Code|  Area|Item Code|  Item|Element Code|             Element|Year Code|Year|       Unit|       Value|Flag|
+---------+------+---------+------+------------+--------------------+---------+----+-----------+------------+----+
|       21|Brazil|      515|Apples|         152|Gross Production ...|     1961|1961|1000 Int. $| 9422.000000|  Fc|
|       21|Brazil|      515|Apples|         152|Gross Production ...|     1962|1962|1000 Int. $|10667.000000|  Fc|
|       21|Brazil|      515|Apples|         152|Gross Production ...|     1963|1963|1000 Int. $|10970.000000|  Fc|
|       21|Brazil|      515|Apples|         152|Gross Production ...|     1964|1964|1000 Int. $| 9987.000000|  Fc|
|       21|Brazil|      515|Apples|         152|Gross Production ...|     1965|1965|1000 Int. $|19062.000000|  Fc|
|       21|Brazil|      515|Apples|         152|Gross Production ...|     1966|1

In [20]:
dataframe_brazil[dataframe_brazil['Item'].isin(['Carne, Gado','Soja'])].show()

+---------+------+---------+-----------+------------+--------------------+---------+----+--------+---------------+----+
|Area Code|  Area|Item Code|       Item|Element Code|             Element|Year Code|Year|    Unit|          Value|Flag|
+---------+------+---------+-----------+------------+--------------------+---------+----+--------+---------------+----+
|       21|Brazil|      867|Carne, Gado|          56|Gross Production ...|     1991|1991|1000 SLC|     945.000000|  Fc|
|       21|Brazil|      867|Carne, Gado|          56|Gross Production ...|     1992|1992|1000 SLC|    9562.000000|  Fc|
|       21|Brazil|      867|Carne, Gado|          56|Gross Production ...|     1993|1993|1000 SLC|  229422.000000|  Fc|
|       21|Brazil|      867|Carne, Gado|          56|Gross Production ...|     1994|1994|1000 SLC| 5916672.000000|  Fc|
|       21|Brazil|      867|Carne, Gado|          56|Gross Production ...|     1995|1995|1000 SLC| 8782288.000000|  Fc|
|       21|Brazil|      867|Carne, Gado|

In [18]:
dataframe_brazil.filter("Item == 'Soja'").show()

+---------+------+---------+----+------------+--------------------+---------+----+-----------+--------------+----+
|Area Code|  Area|Item Code|Item|Element Code|             Element|Year Code|Year|       Unit|         Value|Flag|
+---------+------+---------+----+------------+--------------------+---------+----+-----------+--------------+----+
|       21|Brazil|      236|Soja|         152|Gross Production ...|     1961|1961|1000 Int. $| 103834.000000|  Fc|
|       21|Brazil|      236|Soja|         152|Gross Production ...|     1962|1962|1000 Int. $| 132017.000000|  Fc|
|       21|Brazil|      236|Soja|         152|Gross Production ...|     1963|1963|1000 Int. $| 123503.000000|  Fc|
|       21|Brazil|      236|Soja|         152|Gross Production ...|     1964|1964|1000 Int. $| 116612.000000|  Fc|
|       21|Brazil|      236|Soja|         152|Gross Production ...|     1965|1965|1000 Int. $| 200096.000000|  Fc|
|       21|Brazil|      236|Soja|         152|Gross Production ...|     1966|196

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

dataframe_brazil_pyspark = spark.createDataFrame(dataframe_brazil[['Area','Item','Year','Value']])
_df = dataframe_brazil_pyspark.select(
    col('Area').alias('AREA'),
    col('Item').alias('ITEM'),
    col('Year').cast(IntegerType()).alias('YEAR'),
    col('Value').cast(FloatType()).alias('VALUE')
)

_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-tratados/brazil_tratados_from_world_agriculture')

In [ ]:
dataframe = spark.read\
            .csv(f"{S3_address}/dados-tratados/brazil_tratados_from_world_agriculture")

In [ ]:
dataframe = dataframe.withColumnRenamed("_c0", "Regiao")
dataframe = dataframe.withColumnRenamed("_c1", "Produto")
dataframe = dataframe.withColumnRenamed("_c2", "Ano")
dataframe = dataframe.withColumnRenamed("_c3", "valor $")
dataframe.filter(col("Produto") == "Soja").show()

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import DecimalType

_df = dataframe.select(
    col('Regiao').alias('regiao'),
    col('valor $').cast(DecimalType(10,3)).alias('valor'),
    col('produto').alias('produto'),
    col('ano').cast(IntegerType()).alias('ano')
)
_df
_df.write \
   .mode('overwrite') \
   .csv(f'{S3_address}/dados-clientes/brazil_tratados_from_world_agriculture')

_df.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://poupay-grupo05.database.windows.net;databaseName=poupay;') \
 .option('dbtable', 'tblSoybeanData') \
 .option('user', user_sql ) \
 .option('password', password_sql) \
 .save()